In [1]:
import sys
import io
import time
import requests
import datetime
import numpy as np
import pandas as pd

sys.path.append('../')
from src.util.functions import Functions

# Disable warning
pd.options.mode.chained_assignment = None 

level = 'regioni'

path_data = '../data'
path_csv = '{path_data}/csv'.format(path_data=path_data)

csv_out_name = '{path_csv}/covid19-{level}.csv'.format(path_csv=path_csv, level=level)
pop_regions_name = '{path_csv}/popolazione-regioni.csv'.format(path_csv=path_csv)
icu_regions_name = '{path_csv}/terapie-intensive.csv'.format(path_csv=path_csv)
regions_name = '{path_csv}/regioni.csv'.format(path_csv=path_csv)

In [2]:
url_cum = Functions.get_url_cum(level)
url_cum

'https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-regioni/dpc-covid19-ita-regioni.csv'

In [3]:
df_raw_r = pd.read_csv(url_cum)
df_raw_r

,data,stato,codice_regione,denominazione_regione,lat,long,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,...,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,note
0,2020-02-24T18:00:00,ITA,13,Abruzzo,42.351222,13.398438,0,0,0,0,...,0,0,0,0,NaN,NaN,0,5,NaN,NaN
1,2020-02-24T18:00:00,ITA,17,Basilicata,40.639471,15.805148,0,0,0,0,...,0,0,0,0,NaN,NaN,0,0,NaN,NaN
2,2020-02-24T18:00:00,ITA,18,Calabria,38.905976,16.594402,0,0,0,0,...,0,0,0,0,NaN,NaN,0,1,NaN,NaN
3,2020-02-24T18:00:00,ITA,15,Campania,40.839566,14.250850,0,0,0,0,...,0,0,0,0,NaN,NaN,0,10,NaN,NaN
4,2020-02-24T18:00:00,ITA,8,Emilia-Romagna,44.494367,11.341721,10,2,12,6,...,0,18,0,0,NaN,NaN,18,148,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5896,2020-11-30T17:00:00,ITA,19,Sicilia,38.115697,13.362357,1547,226,1773,38851,...,140,1138,21507,1555,38536.0,25150.0,63686,959923,654885.0,NaN
5897,2020-11-30T17:00:00,ITA,9,Toscana,43.769231,11.255889,1584,278,1862,40164,...,-1518,893,58774,2641,78969.0,24472.0,103441,1568208,958389.0,effettuati 84798 tamponi rapidi antigenici di...
5898,2020-11-30T17:00:00,ITA,10,Umbria,43.106758,12.388247,366,65,431,7804,...,-208,68,15166,407,6620.0,17188.0,23808,416530,224790.0,"Si fa presente come dei 13 decessi odierni, 6 ..."
5899,2020-11-30T17:00:00,ITA,2,Valle d'Aosta,45.737503,7.320149,124,11,135,1182,...,-133,47,4878,315,5798.0,712.0,6510,60435,33903.0,NaN


In [4]:
# Drop useless columns
cols_to_drop = ['stato', 'lat', 'long', 'note']
df_rd = df_raw_r.drop(cols_to_drop, axis=1)
df_rd

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati
0,2020-02-24T18:00:00,13,Abruzzo,0,0,0,0,0,0,0,0,0,NaN,NaN,0,5,NaN
1,2020-02-24T18:00:00,17,Basilicata,0,0,0,0,0,0,0,0,0,NaN,NaN,0,0,NaN
2,2020-02-24T18:00:00,18,Calabria,0,0,0,0,0,0,0,0,0,NaN,NaN,0,1,NaN
3,2020-02-24T18:00:00,15,Campania,0,0,0,0,0,0,0,0,0,NaN,NaN,0,10,NaN
4,2020-02-24T18:00:00,8,Emilia-Romagna,10,2,12,6,18,0,18,0,0,NaN,NaN,18,148,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5896,2020-11-30T17:00:00,19,Sicilia,1547,226,1773,38851,40624,140,1138,21507,1555,38536.0,25150.0,63686,959923,654885.0
5897,2020-11-30T17:00:00,9,Toscana,1584,278,1862,40164,42026,-1518,893,58774,2641,78969.0,24472.0,103441,1568208,958389.0
5898,2020-11-30T17:00:00,10,Umbria,366,65,431,7804,8235,-208,68,15166,407,6620.0,17188.0,23808,416530,224790.0
5899,2020-11-30T17:00:00,2,Valle d'Aosta,124,11,135,1182,1317,-133,47,4878,315,5798.0,712.0,6510,60435,33903.0


In [5]:
last_date_str = df_rd.iloc[-1].data.split('T')[0]
last_date = datetime.datetime.strptime(last_date_str, '%Y-%m-%d')
prev_date = Functions.get_previous_date(last_date)
prev_date_str = datetime.datetime.strftime(prev_date, '%Y-%m-%d')
last_date_str, prev_date_str

('2020-11-30', '2020-11-29')

In [6]:
# Current state S
df_raw_r_1 = df_rd[df_rd.data.str.startswith(last_date_str)].sort_values(by='codice_regione').reset_index(drop=True)
df_raw_r_1

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati
0,2020-11-30T17:00:00,1,Piemonte,4737,388,5125,69818,74943,-2041,1185,86334,6239,70013.0,97503.0,167516,1558737,961035.0
1,2020-11-30T17:00:00,2,Valle d'Aosta,124,11,135,1182,1317,-133,47,4878,315,5798.0,712.0,6510,60435,33903.0
2,2020-11-30T17:00:00,3,Lombardia,7433,906,8339,117069,125408,-7219,1929,260528,21855,299979.0,107812.0,407791,4072678,2349286.0
3,2020-11-30T17:00:00,5,Veneto,2460,320,2780,77885,80665,954,2003,61216,3711,36847.0,108745.0,145592,2791439,1062810.0
4,2020-11-30T17:00:00,6,Friuli Venezia Giulia,633,55,688,13827,14515,206,575,15297,838,26263.0,4387.0,30650,720175,281701.0
5,2020-11-30T17:00:00,7,Liguria,997,113,1110,11106,12216,-390,236,37081,2387,34778.0,16906.0,51684,600386,290739.0
6,2020-11-30T17:00:00,8,Emilia-Romagna,2755,249,3004,68730,71734,1505,2041,45586,5753,75472.0,47601.0,123073,2120489,1079854.0
7,2020-11-30T17:00:00,9,Toscana,1584,278,1862,40164,42026,-1518,893,58774,2641,78969.0,24472.0,103441,1568208,958389.0
8,2020-11-30T17:00:00,10,Umbria,366,65,431,7804,8235,-208,68,15166,407,6620.0,17188.0,23808,416530,224790.0
9,2020-11-30T17:00:00,11,Marche,585,87,672,17244,17916,220,252,10626,1264,23205.0,6601.0,29806,411597,239816.0


In [7]:
# State S - 1
df_raw_r_0 = df_rd[df_rd.data.str.startswith(prev_date_str)].sort_values(by='codice_regione').reset_index(drop=True)
df_raw_r_0

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati
0,2020-11-29T17:00:00,1,Piemonte,4734,385,5119,71865,76984,907,2021,83193,6154,69743.0,96588.0,166331,1546447,956383.0
1,2020-11-29T17:00:00,2,Valle d'Aosta,138,13,151,1299,1450,-15,47,4702,311,5756.0,707.0,6463,59860,33672.0
2,2020-11-29T17:00:00,3,Lombardia,7400,907,8307,124320,132627,2312,3203,251588,21647,298591.0,107271.0,405862,4055691,2343093.0
3,2020-11-29T17:00:00,5,Veneto,2442,311,2753,76958,79711,1139,2617,60201,3677,36506.0,107083.0,143589,2782454,1059827.0
4,2020-11-29T17:00:00,6,Friuli Venezia Giulia,620,57,677,13632,14309,313,680,14949,817,25753.0,4322.0,30075,713195,280065.0
5,2020-11-29T17:00:00,7,Liguria,970,111,1081,11525,12606,-119,437,36462,2380,35049.0,16399.0,51448,598285,290019.0
6,2020-11-29T17:00:00,8,Emilia-Romagna,2668,245,2913,67316,70229,446,1850,45089,5714,74620.0,46412.0,121032,2109497,1074568.0
7,2020-11-29T17:00:00,9,Toscana,1590,278,1868,41676,43544,-1528,908,56403,2601,78463.0,24085.0,102548,1557825,954647.0
8,2020-11-29T17:00:00,10,Umbria,355,65,420,8023,8443,-210,310,14903,394,6595.0,17145.0,23740,416050,224573.0
9,2020-11-29T17:00:00,11,Marche,598,85,683,17013,17696,453,518,10603,1255,29554.0,0.0,29554,409684,238861.0


In [8]:
# New df regions state S with Trentino-Alto Adige
df_r_1 = Functions.replace_bt_with_taa(df_raw_r_1)
df_r_1.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati
0,2020-11-30T17:00:00,1,Piemonte,4737,388,5125,69818,74943,-2041,1185,86334,6239,70013.0,97503.0,167516,1558737,961035.0
1,2020-11-30T17:00:00,2,Valle d'Aosta,124,11,135,1182,1317,-133,47,4878,315,5798.0,712.0,6510,60435,33903.0
2,2020-11-30T17:00:00,3,Lombardia,7433,906,8339,117069,125408,-7219,1929,260528,21855,299979.0,107812.0,407791,4072678,2349286.0
3,2020-11-30T17:00:00,4,Trentino-Alto Adige,843,81,924,12846,13770,114,359,24555,1185,31657.0,7853.0,39510,687213,284275.0
4,2020-11-30T17:00:00,5,Veneto,2460,320,2780,77885,80665,954,2003,61216,3711,36847.0,108745.0,145592,2791439,1062810.0
5,2020-11-30T17:00:00,6,Friuli Venezia Giulia,633,55,688,13827,14515,206,575,15297,838,26263.0,4387.0,30650,720175,281701.0
6,2020-11-30T17:00:00,7,Liguria,997,113,1110,11106,12216,-390,236,37081,2387,34778.0,16906.0,51684,600386,290739.0
7,2020-11-30T17:00:00,8,Emilia-Romagna,2755,249,3004,68730,71734,1505,2041,45586,5753,75472.0,47601.0,123073,2120489,1079854.0
8,2020-11-30T17:00:00,9,Toscana,1584,278,1862,40164,42026,-1518,893,58774,2641,78969.0,24472.0,103441,1568208,958389.0
9,2020-11-30T17:00:00,10,Umbria,366,65,431,7804,8235,-208,68,15166,407,6620.0,17188.0,23808,416530,224790.0


In [9]:
# New df regions state S - 1 with Trentino-Alto Adige
df_r_0 = Functions.replace_bt_with_taa(df_raw_r_0)
df_r_0.sort_values(by='codice_regione').reset_index()

,index,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati
0,0,2020-11-29T17:00:00,1,Piemonte,4734,385,5119,71865,76984,907,2021,83193,6154,69743.0,96588.0,166331,1546447,956383.0
1,1,2020-11-29T17:00:00,2,Valle d'Aosta,138,13,151,1299,1450,-15,47,4702,311,5756.0,707.0,6463,59860,33672.0
2,2,2020-11-29T17:00:00,3,Lombardia,7400,907,8307,124320,132627,2312,3203,251588,21647,298591.0,107271.0,405862,4055691,2343093.0
3,19,2020-11-29T17:00:00,4,Trentino-Alto Adige,824,80,904,12752,13656,-35,702,24324,1171,31413.0,7738.0,39151,685474,283658.0
4,3,2020-11-29T17:00:00,5,Veneto,2442,311,2753,76958,79711,1139,2617,60201,3677,36506.0,107083.0,143589,2782454,1059827.0
5,4,2020-11-29T17:00:00,6,Friuli Venezia Giulia,620,57,677,13632,14309,313,680,14949,817,25753.0,4322.0,30075,713195,280065.0
6,5,2020-11-29T17:00:00,7,Liguria,970,111,1081,11525,12606,-119,437,36462,2380,35049.0,16399.0,51448,598285,290019.0
7,6,2020-11-29T17:00:00,8,Emilia-Romagna,2668,245,2913,67316,70229,446,1850,45089,5714,74620.0,46412.0,121032,2109497,1074568.0
8,7,2020-11-29T17:00:00,9,Toscana,1590,278,1868,41676,43544,-1528,908,56403,2601,78463.0,24085.0,102548,1557825,954647.0
9,8,2020-11-29T17:00:00,10,Umbria,355,65,420,8023,8443,-210,310,14903,394,6595.0,17145.0,23740,416050,224573.0


In [10]:
df_r = df_r_1.copy()
# Merge with population data
df_pop_r = pd.read_csv(pop_regions_name)
df_m_r_1 = df_r.merge(df_pop_r, on='denominazione_regione')
df_m_r_1.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,popolazione
0,2020-11-30T17:00:00,1,Piemonte,4737,388,5125,69818,74943,-2041,1185,86334,6239,70013.0,97503.0,167516,1558737,961035.0,4341375
1,2020-11-30T17:00:00,2,Valle d'Aosta,124,11,135,1182,1317,-133,47,4878,315,5798.0,712.0,6510,60435,33903.0,125501
2,2020-11-30T17:00:00,3,Lombardia,7433,906,8339,117069,125408,-7219,1929,260528,21855,299979.0,107812.0,407791,4072678,2349286.0,10103969
3,2020-11-30T17:00:00,4,Trentino-Alto Adige,843,81,924,12846,13770,114,359,24555,1185,31657.0,7853.0,39510,687213,284275.0,1074819
4,2020-11-30T17:00:00,5,Veneto,2460,320,2780,77885,80665,954,2003,61216,3711,36847.0,108745.0,145592,2791439,1062810.0,4907704
5,2020-11-30T17:00:00,6,Friuli Venezia Giulia,633,55,688,13827,14515,206,575,15297,838,26263.0,4387.0,30650,720175,281701.0,1211357
6,2020-11-30T17:00:00,7,Liguria,997,113,1110,11106,12216,-390,236,37081,2387,34778.0,16906.0,51684,600386,290739.0,1543127
7,2020-11-30T17:00:00,8,Emilia-Romagna,2755,249,3004,68730,71734,1505,2041,45586,5753,75472.0,47601.0,123073,2120489,1079854.0,4467118
8,2020-11-30T17:00:00,9,Toscana,1584,278,1862,40164,42026,-1518,893,58774,2641,78969.0,24472.0,103441,1568208,958389.0,3722729
9,2020-11-30T17:00:00,10,Umbria,366,65,431,7804,8235,-208,68,15166,407,6620.0,17188.0,23808,416530,224790.0,880285


In [11]:
# Merge with icu data
df_icu_r = pd.read_csv(icu_regions_name)
df_m_r_2 = df_m_r_1.merge(df_icu_r, on='denominazione_regione')
df_m_r_2.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,popolazione,totale_posti_terapia_intensiva
0,2020-11-30T17:00:00,1,Piemonte,4737,388,5125,69818,74943,-2041,1185,86334,6239,70013.0,97503.0,167516,1558737,961035.0,4341375,575
1,2020-11-30T17:00:00,2,Valle d'Aosta,124,11,135,1182,1317,-133,47,4878,315,5798.0,712.0,6510,60435,33903.0,125501,20
2,2020-11-30T17:00:00,3,Lombardia,7433,906,8339,117069,125408,-7219,1929,260528,21855,299979.0,107812.0,407791,4072678,2349286.0,10103969,1036
3,2020-11-30T17:00:00,4,Trentino-Alto Adige,843,81,924,12846,13770,114,359,24555,1185,31657.0,7853.0,39510,687213,284275.0,1074819,106
4,2020-11-30T17:00:00,5,Veneto,2460,320,2780,77885,80665,954,2003,61216,3711,36847.0,108745.0,145592,2791439,1062810.0,4907704,1016
5,2020-11-30T17:00:00,6,Friuli Venezia Giulia,633,55,688,13827,14515,206,575,15297,838,26263.0,4387.0,30650,720175,281701.0,1211357,180
6,2020-11-30T17:00:00,7,Liguria,997,113,1110,11106,12216,-390,236,37081,2387,34778.0,16906.0,51684,600386,290739.0,1543127,209
7,2020-11-30T17:00:00,8,Emilia-Romagna,2755,249,3004,68730,71734,1505,2041,45586,5753,75472.0,47601.0,123073,2120489,1079854.0,4467118,634
8,2020-11-30T17:00:00,9,Toscana,1584,278,1862,40164,42026,-1518,893,58774,2641,78969.0,24472.0,103441,1568208,958389.0,3722729,523
9,2020-11-30T17:00:00,10,Umbria,366,65,431,7804,8235,-208,68,15166,407,6620.0,17188.0,23808,416530,224790.0,880285,117


In [12]:
# Copy last merged df into df_r
df_r = df_m_r_2.copy()

In [13]:
# Add columns
df_r['variazione_ricoverati_con_sintomi'] = df_r_1['ricoverati_con_sintomi'] - df_r_0['ricoverati_con_sintomi']
df_r['variazione_terapia_intensiva'] = df_r_1['terapia_intensiva'] - df_r_0['terapia_intensiva']
df_r['variazione_totale_ospedalizzati'] = df_r_1['totale_ospedalizzati'] - df_r_0['totale_ospedalizzati']
df_r['posti_disponibili_terapia_intensiva'] = df_r['totale_posti_terapia_intensiva'] - df_r['terapia_intensiva']
df_r['saturazione_terapia_intensiva'] = ((df_r_1['terapia_intensiva'] / df_r['totale_posti_terapia_intensiva']) * 100).round(2)
df_r['variazione_isolamento_domiciliare'] = df_r_1['isolamento_domiciliare'] - df_r_0['isolamento_domiciliare']
df_r['variazione_nuovi_positivi'] = df_r_1['nuovi_positivi'] - df_r_0['nuovi_positivi']
df_r['nuovi_tamponi'] = df_r_1['tamponi'] - df_r_0['tamponi']
df_r['nuovi_casi_testati'] = df_r_1['casi_testati'] - df_r_0['casi_testati']
df_r['nuovi_deceduti'] = df_r_1['deceduti'] - df_r_0['deceduti']
df_r['nuovi_dimessi_guariti'] = df_r_1['dimessi_guariti'] - df_r_0['dimessi_guariti']
df_r['np_su_nt'] = ((df_r['nuovi_positivi'] / df_r['nuovi_tamponi']) * 100).round(2)
df_r['np_su_p'] = ((df_r['nuovi_positivi'] / df_r['popolazione']) * 100).round(5)
df_r['tc_su_p'] = ((df_r['totale_casi'] / df_r['popolazione']) * 100).round(2)

df_r = df_r.sort_values(by='nuovi_positivi', ascending=False).reset_index(drop=True)
df_r

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,saturazione_terapia_intensiva,variazione_isolamento_domiciliare,variazione_nuovi_positivi,nuovi_tamponi,nuovi_casi_testati,nuovi_deceduti,nuovi_dimessi_guariti,np_su_nt,np_su_p,tc_su_p
0,2020-11-30T17:00:00,8,Emilia-Romagna,2755,249,3004,68730,71734,1505,2041,...,39.27,1414,191,10992,5286.0,39,497,18.57,0.04569,2.76
1,2020-11-30T17:00:00,5,Veneto,2460,320,2780,77885,80665,954,2003,...,31.50,927,-614,8985,2983.0,34,1015,22.29,0.04081,2.97
2,2020-11-30T17:00:00,3,Lombardia,7433,906,8339,117069,125408,-7219,1929,...,87.45,-7251,-1274,16987,6193.0,208,8940,11.36,0.01909,4.04
3,2020-11-30T17:00:00,15,Campania,2141,182,2323,102204,104527,-259,1626,...,36.04,-234,-396,14286,7143.0,42,1843,11.38,0.02810,2.68
4,2020-11-30T17:00:00,12,Lazio,3384,350,3734,86467,90201,489,1589,...,41.32,468,-404,20008,15023.0,39,1061,7.94,0.02709,2.04
5,2020-11-30T17:00:00,1,Piemonte,4737,388,5125,69818,74943,-2041,1185,...,67.48,-2047,-836,12290,4652.0,85,3141,9.64,0.02730,3.86
6,2020-11-30T17:00:00,19,Sicilia,1547,226,1773,38851,40624,140,1138,...,38.44,130,114,8602,4965.0,49,949,13.23,0.02290,1.28
7,2020-11-30T17:00:00,16,Puglia,1692,209,1901,36871,38772,528,1102,...,56.64,514,195,4151,2979.0,30,544,26.55,0.02749,1.36
8,2020-11-30T17:00:00,9,Toscana,1584,278,1862,40164,42026,-1518,893,...,53.15,-1512,-15,10383,3742.0,40,2371,8.60,0.02399,2.78
9,2020-11-30T17:00:00,6,Friuli Venezia Giulia,633,55,688,13827,14515,206,575,...,30.56,195,-105,6980,1636.0,21,348,8.24,0.04747,2.53


In [14]:
# Write pre-processed data
df_r.to_csv(csv_out_name, index=False)

In [15]:
# ICU Stats
df_r[['denominazione_regione', 'terapia_intensiva', 'posti_disponibili_terapia_intensiva', 'totale_posti_terapia_intensiva', 'saturazione_terapia_intensiva']] \
    .sort_values(by='saturazione_terapia_intensiva', ascending=False).reset_index(drop=True)

,denominazione_regione,terapia_intensiva,posti_disponibili_terapia_intensiva,totale_posti_terapia_intensiva,saturazione_terapia_intensiva
0,Lombardia,906,130,1036,87.45
1,Trentino-Alto Adige,81,25,106,76.42
2,Piemonte,388,187,575,67.48
3,Marche,87,56,143,60.84
4,Puglia,209,160,369,56.64
5,Umbria,65,52,117,55.56
6,Valle d'Aosta,11,9,20,55.00
7,Liguria,113,96,209,54.07
8,Toscana,278,245,523,53.15
9,Abruzzo,76,72,148,51.35


In [16]:
def add_cols(df):
    df['variazione_terapia_intensiva'] = df['terapia_intensiva'].diff()
    df['variazione_ricoverati_con_sintomi'] = df['ricoverati_con_sintomi'].diff()
    df['variazione_totale_ospedalizzati'] = df['totale_ospedalizzati'].diff()
    df['nuovi_dimessi_guariti'] = df['dimessi_guariti'].diff()
    df['nuovi_deceduti'] = df['deceduti'].diff()
    df['nuovi_tamponi'] = df['tamponi'].diff()
    df['nuovi_casi_testati'] = df['casi_testati'].diff()
    df['variazione_isolamento_domiciliare'] = df['isolamento_domiciliare'].diff()
    df = df.iloc[1:]
    df['ti_su_to'] = ((df['terapia_intensiva'] / df['totale_ospedalizzati']).replace(np.inf, 0) * 100).round(2)
    df['to_su_tp'] = ((df['totale_ospedalizzati'] / df['totale_positivi']).replace(np.inf, 0) * 100).round(2)
    df['np_su_nt'] = ((df['nuovi_positivi'] / df['nuovi_tamponi']).replace(np.inf, 0) * 100).round(2)
    df['np_su_nct'] = ((df['nuovi_positivi'] / df['nuovi_casi_testati']).replace(np.inf, 0) * 100).round(2)
    df['var_ti_perc'] = ((df['variazione_terapia_intensiva'] / (df['terapia_intensiva'] - df['variazione_terapia_intensiva'])).replace(np.inf, 0) * 100).round(2)
    df['var_pos_perc'] = ((df['variazione_totale_positivi'] / (df['totale_positivi'] - df['variazione_totale_positivi'])).replace(np.inf, 0) * 100).round(2)
    return df.fillna(0)

In [17]:
# df_rd.sort_values(by='codice_regione').reset_index(drop=True)
df_rn = pd.read_csv(regions_name)
for region_code in df_rn.codice_regione:
    if region_code != 4: # TAA
        df_cur_region = df_rd[df_rd.codice_regione == region_code]
        region_name = df_cur_region.denominazione_regione.iloc[0].lower()
        csv_out_region_name = '{path_csv}/regioni/covid19-{region_name}.csv'.format(path_csv=path_csv, region_name=region_name)
        df_cur_region = add_cols(df_cur_region)
        df_cur_region.to_csv(csv_out_region_name, index=False)
        print('Aggiornato {file}'.format(file=csv_out_region_name))

Aggiornato ../data/csv/regioni/covid19-piemonte.csv
Aggiornato ../data/csv/regioni/covid19-valle d'aosta.csv
Aggiornato ../data/csv/regioni/covid19-lombardia.csv
Aggiornato ../data/csv/regioni/covid19-veneto.csv
Aggiornato ../data/csv/regioni/covid19-friuli venezia giulia.csv
Aggiornato ../data/csv/regioni/covid19-liguria.csv
Aggiornato ../data/csv/regioni/covid19-emilia-romagna.csv
Aggiornato ../data/csv/regioni/covid19-toscana.csv
Aggiornato ../data/csv/regioni/covid19-umbria.csv
Aggiornato ../data/csv/regioni/covid19-marche.csv
Aggiornato ../data/csv/regioni/covid19-lazio.csv
Aggiornato ../data/csv/regioni/covid19-abruzzo.csv
Aggiornato ../data/csv/regioni/covid19-molise.csv
Aggiornato ../data/csv/regioni/covid19-campania.csv
Aggiornato ../data/csv/regioni/covid19-puglia.csv
Aggiornato ../data/csv/regioni/covid19-basilicata.csv
Aggiornato ../data/csv/regioni/covid19-calabria.csv
Aggiornato ../data/csv/regioni/covid19-sicilia.csv
Aggiornato ../data/csv/regioni/covid19-sardegna.csv


In [18]:
df_tb = df_rd[df_rd.codice_regione > 20].reset_index(drop=True)
df_tb

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati
0,2020-02-24T18:00:00,21,P.A. Bolzano,0,0,0,0,0,0,0,0,0,NaN,NaN,0,1,NaN
1,2020-02-24T18:00:00,22,P.A. Trento,0,0,0,0,0,0,0,0,0,NaN,NaN,0,3,NaN
2,2020-02-25T18:00:00,21,P.A. Bolzano,1,0,1,0,1,1,1,0,0,NaN,NaN,1,1,NaN
3,2020-02-25T18:00:00,22,P.A. Trento,0,0,0,0,0,0,0,0,0,NaN,NaN,0,3,NaN
4,2020-02-26T18:00:00,21,P.A. Bolzano,1,0,1,0,1,0,0,0,0,NaN,NaN,1,1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
557,2020-11-28T17:00:00,22,P.A. Trento,404,42,446,1687,2133,-176,219,12530,629,7742.0,7550.0,15292,371382,134489.0
558,2020-11-29T17:00:00,21,P.A. Bolzano,420,36,456,10881,11337,-221,437,11727,530,23594.0,0.0,23594,310397,148650.0
559,2020-11-29T17:00:00,22,P.A. Trento,404,44,448,1871,2319,186,265,12597,641,7819.0,7738.0,15557,375077,135008.0
560,2020-11-30T17:00:00,21,P.A. Bolzano,428,37,465,10857,11322,-15,183,11924,531,23777.0,0.0,23777,311399,149001.0


In [19]:
# Initialize df_taa
df_taa = Functions.replace_bt_with_taa(df_tb[:2])
for i in range(2, df_tb.shape[0], 2):
    df_taa = df_taa.append(Functions.replace_bt_with_taa(df_tb[i:i+2]), ignore_index=True)
df_taa = add_cols(df_taa)
df_taa

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,nuovi_deceduti,nuovi_tamponi,nuovi_casi_testati,variazione_isolamento_domiciliare,ti_su_to,to_su_tp,np_su_nt,np_su_nct,var_ti_perc,var_pos_perc
1,2020-02-25T18:00:00,4,Trentino-Alto Adige,1,0,1,0,1,1,1,...,0.0,0.0,0.0,0.0,0.00,100.00,0.00,0.00,0.00,0.00
2,2020-02-26T18:00:00,4,Trentino-Alto Adige,1,0,1,0,1,0,0,...,0.0,0.0,0.0,0.0,0.00,100.00,0.00,0.00,0.00,0.00
3,2020-02-27T18:00:00,4,Trentino-Alto Adige,1,0,1,0,1,0,0,...,0.0,30.0,0.0,0.0,0.00,100.00,0.00,0.00,0.00,0.00
4,2020-02-28T18:00:00,4,Trentino-Alto Adige,1,0,1,0,1,0,0,...,0.0,0.0,0.0,0.0,0.00,100.00,0.00,0.00,0.00,0.00
5,2020-02-29T17:00:00,4,Trentino-Alto Adige,1,0,1,0,1,0,0,...,0.0,25.0,0.0,0.0,0.00,100.00,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
276,2020-11-26T17:00:00,4,Trentino-Alto Adige,851,83,934,12878,13812,-235,589,...,20.0,6425.0,1807.0,-224.0,8.89,6.76,9.17,32.60,1.22,-1.67
277,2020-11-27T17:00:00,4,Trentino-Alto Adige,822,86,908,12884,13792,-20,639,...,19.0,5855.0,1903.0,6.0,9.47,6.58,10.91,33.58,3.61,-0.14
278,2020-11-28T17:00:00,4,Trentino-Alto Adige,813,78,891,12800,13691,-101,713,...,9.0,6342.0,2208.0,-84.0,8.75,6.51,11.24,32.29,-9.30,-0.73
279,2020-11-29T17:00:00,4,Trentino-Alto Adige,824,80,904,12752,13656,-35,702,...,19.0,7177.0,1569.0,-48.0,8.85,6.62,9.78,44.74,2.56,-0.26


In [20]:
region_name = df_taa.denominazione_regione.iloc[0].lower()
csv_out_region_name = '{path_csv}/regioni/covid19-{region_name}.csv'.format(path_csv=path_csv, region_name=region_name)
df_taa.to_csv(csv_out_region_name, index=False)
print('Aggiornato {file}'.format(file=csv_out_region_name))

Aggiornato ../data/csv/regioni/covid19-trentino-alto adige.csv
